In [14]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import joblib
import pandas as pd

# Load the dataset
data = pd.read_csv('portScan.csv')
X = data[['Flow Bytes/s', 'Total Length of Fwd Packets', 'Fwd IAT Total', 'Flow Duration']]
# y = data['Label']

# data.fillna(0)
print("Size of dataset=",data.size)
attack_or_not=[]
for i in data['Label']: #it changes the normal label to "1" and the attack tag to "0" for use in the machine learning algorithm
    if i =="BENIGN":
        attack_or_not.append(1)
    else:
        attack_or_not.append(0)     
        # attack_or_not.append(1) 
data['Label']=attack_or_not

y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the TensorFlow model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]

accuracy = model.evaluate(X_test, y_test)[1]
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('tensorflow_model')

# Save the metrics
metrics = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1-score': f1}
joblib.dump(metrics, 'metrics.pkl')


Size of dataset= 24344255
Epoch 1/10
7161/7161 [==============================] - 9s 1ms/step - loss: 8606.3359 - accuracy: 0.4954
Epoch 2/10
7161/7161 [==============================] - 9s 1ms/step - loss: 7539.3115 - accuracy: 0.4995
Epoch 3/10
7161/7161 [==============================] - 9s 1ms/step - loss: 5401.8184 - accuracy: 0.5347
Epoch 4/10
7161/7161 [==============================] - 9s 1ms/step - loss: 4641.9155 - accuracy: 0.5544
Epoch 5/10
7161/7161 [==============================] - 9s 1ms/step - loss: 4132.1201 - accuracy: 0.5716
Epoch 6/10
7161/7161 [==============================] - 9s 1ms/step - loss: 2379.4509 - accuracy: 0.6270
Epoch 7/10
7161/7161 [==============================] - 9s 1ms/step - loss: 2190.8511 - accuracy: 0.6519
Epoch 8/10
7161/7161 [==============================] - 9s 1ms/step - loss: 1682.0312 - accuracy: 0.6951
Epoch 9/10
7161/7161 [==============================] - 9s 1ms/step - loss: 1061.2218 - accuracy: 0.7505
Epoch 10/10
1791/1791 [======

INFO:tensorflow:Assets written to: tensorflow_model/assets


INFO:tensorflow:Assets written to: tensorflow_model/assets


['metrics.pkl']

In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np

model = tf.keras.models.load_model('tensorflow_model')
data = pd.read_csv('attacks/Test/benign_4.csv')
X_new = data[['Flow Bytes/s', 'Total Length of Fwd Packets', 'Fwd IAT Total', 'Flow Duration']]
y_pred = model.predict(X_new)

# Generating network health report
print("\n\n")
print(110*"*")
# print(f"\nReport For  File\n")
print(y_pred)
if np.mean(y_pred) < 0.5:
    print("Malicious Network found, Disconnect quickly")
else:
    print("Safe Network")



5/5 [==============================] - 0s 2ms/step



**************************************************************************************************************
[[1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [0.03437895]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [       nan]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [       nan]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [0.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.        ]
 [1.     

In [15]:
import tensorflow as tf

# Load the saved TensorFlow model
model = tf.keras.models.load_model('tensorflow_model')

# Convert the TensorFlow model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('tensorflow_model.tflite', 'wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /tmp/tmpcm3xy8pt/assets


INFO:tensorflow:Assets written to: /tmp/tmpcm3xy8pt/assets
2023-06-20 10:23:58.996628: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-06-20 10:23:58.996665: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-06-20 10:23:58.997735: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpcm3xy8pt
2023-06-20 10:23:58.998385: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-06-20 10:23:58.998398: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpcm3xy8pt
2023-06-20 10:23:59.000168: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-06-20 10:23:59.000747: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-06-20 10:23:59.020399: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

In [21]:
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import joblib
import pandas as pd
import tensorflow as tf

# Load the dataset
data = pd.read_csv('portScan_test.csv')
X = data[['Flow Bytes/s', 'Total Length of Fwd Packets', 'Fwd IAT Total', 'Flow Duration']]

attack_or_not=[]
for i in data['Label']:
    if i =="BENIGN":
        attack_or_not.append(1)
    else:
        attack_or_not.append(0)     
data['Label'] = attack_or_not

y = data['Label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data to TensorFlow Dataset format and add a batch operation
train_data = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values)).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))

# Define the TF-DF model
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.CLASSIFICATION)

# Train the model
model.fit(train_data)

# Evaluate the model
y_pred = model.predict(test_data)
y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]

accuracy = model.evaluate(test_data)["accuracy"]
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Save the model
model.save('tfdf_model')

# Save the metrics
metrics = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1-score': f1}
joblib.dump(metrics, 'metrics.pkl')


Use /tmp/tmpvv5uedub as temporary training directory
Reading training dataset...


2023-06-20 11:21:12.387988: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [26]
	 [[{{node Placeholder/_1}}]]


Training dataset read in 0:00:03.355638. Found 26 examples.
Training model...
Model trained in 0:00:00.047682
Compiling model...


[INFO 23-06-20 11:21:15.7715 IST kernel.cc:1242] Loading model from path /tmp/tmpvv5uedub/model/ with prefix cbbf342d17ae4379
[WARNING 23-06-20 11:21:15.7741 IST utils.cc:73] The model does not have any input features i.e. the model is constant and will always return the same prediction.
[INFO 23-06-20 11:21:15.7741 IST decision_forest.cc:660] Model loaded with 300 root(s), 300 node(s), and 0 input feature(s).
[INFO 23-06-20 11:21:15.7742 IST abstract_model.cc:1311] Engine "RandomForestOptPred" built
[INFO 23-06-20 11:21:15.7742 IST kernel.cc:1074] Use fast generic engine
2023-06-20 11:21:15.803921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [26]
	 [[{{node Placeholder/_1}}]]


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


2023-06-20 11:21:18.490818: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [7]
	 [[{{node Placeholder/_1}}]]


ValueError: in user code:

    File "/home/abhi/.local/lib/python3.11/site-packages/tensorflow_decision_forests/keras/core_inference.py", line 458, in run_step  *
        outputs = model.predict_step(data)
    File "/home/abhi/.local/lib/python3.11/site-packages/keras/engine/training.py", line 2111, in predict_step  **
        return self(x, training=False)
    File "/home/abhi/.local/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_filer7icmfzn.py", line 104, in tf__call
        ag__.if_stmt(ag__.ld(self)._semantics is None, if_body_2, else_body_2, get_state_3, set_state_3, ('do_return', 'retval_'), 2)
    File "/tmp/__autograph_generated_filer7icmfzn.py", line 43, in else_body_2
        normalized_inputs = ag__.converted_call(ag__.ld(self)._build_normalized_inputs, (ag__.ld(inputs),), None, fscope)
    File "/tmp/__autograph_generated_file_yqsmxqv.py", lin